# Piano PLayer 🎹

### Imports

In [31]:
import cv2
import os

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

from threading import Thread

from playsound import playsound

import math

### List of Sounds

In [32]:
Sounds = os.listdir("../sounds")

### Functions

In [34]:
def calculate_distance(ponto1, ponto2):
    return math.sqrt((ponto1[0] - ponto2[0])**2 + (ponto1[1] - ponto2[1])**2)

### Piano

In [35]:
class PianoPlayer:
  def __init__(self, num_keys):
    
    if num_keys > 28 and num_keys <= 0:
      Exception("Numero de teclas não aceitavel")
    else:
      self.num_keys = num_keys
      
    self.audio_executed = [False] * num_keys
    
  def read_keys(self):
    for x in range(1, self.num_keys):
      cv2.circle(frame,(0+(x*80),50), 5, (0,0,255), -1)
    
  def play_sound(self, x):
    self.audio_executed[x-1] = True
    playsound(f'C:/Users/andre/OneDrive/Documentos/piano/sounds/{Sounds[int(28 / (2 + self.num_keys - (1*x)))+x]}')
    
  def key_press(self, indicador_x, indicador_y):
    for x in range(1, self.num_keys):
      if calculate_distance((indicador_x, indicador_y), (0+(x*80),50)) <= 5 and not self.audio_executed[x-1]:
        self.audio_executed[x-1] = True
        self.play_sound(x)
        self.audio_executed[x-1] = False
      elif calculate_distance((indicador_x, indicador_y), (0+(x*80),50)) >= 40:
        self.audio_executed[x-1] = False
        

- ##### Key numbers are one unit less than selected

In [36]:
piano = PianoPlayer(7)

### Configure Hands

In [37]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils

### CV2 

In [38]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
  
  ret, frame = cap.read()
  
  if not ret:
      print("Erro ao capturar o frame.")
      break
    
  # Read Keys
  piano.read_keys()
  
  rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  results = hands.process(rgb_frame)

  if results.multi_hand_landmarks and results.multi_handedness:
    
    for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
      
      hand_label = hand_handedness.classification[0].label
      
      h, w, _ = frame.shape
      
      # Point hand
      indicador = hand_landmarks.landmark[8]
      indicador_x, indicador_y = int(indicador.x * w), int(indicador.y * h)

      # Piano Press Key ?
      Thread(target=piano.key_press, args=(indicador_x, indicador_y,), daemon=True).start()
      
      mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
      
  cv2.imshow("MediaPipe Hands - Gestos Especificos", frame) 
  
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

cap.release()
cv2.destroyAllWindows()